In [23]:
%matplotlib inline 

import pandas as pd
from collections import Counter
import functools
import tqdm
import re
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier

# Load Larger LSTM network and generate text
import sys
import numpy
import random

import pandas as pd
import tqdm
import re

import string, nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import numpy as np

from sklearn.metrics import f1_score

from pymorphy2 import MorphAnalyzer
import pymorphy2

%matplotlib inline 


from nltk.collocations import *

import pandas as pd
from collections import Counter
import functools
import tqdm
import re
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from itertools import product

from __future__ import division

from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

import nltk, string
from nltk.tokenize import punkt
from nltk.tokenize import word_tokenize
from nltk.tokenize import WhitespaceTokenizer
from nltk.corpus import stopwords

from pymorphy2 import MorphAnalyzer
import pymorphy2

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import ward, dendrogram
from scipy.cluster import hierarchy
from sklearn.metrics.pairwise import cosine_similarity

**load data**

In [24]:
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
stemmer = nltk.stem.porter.PorterStemmer()

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

In [25]:
def normalize_answer(text):
    return ' '.join(re.findall(r"\w+", text)).lower()

def sentence_to_word(sentences):
    sentences_in_words = list()
    for sentence in sentences:
        sentences_in_words.append(normalize_answer(sentence).split())
    return sentences_in_words

def text_to_sentence(text):
    sentences = text.split(". ")
    return [s.strip() for s in sentences if s.strip() != '']

def get_max_match_sentance(data_row):
    sentences = text_to_sentence(data_row["paragraph"])
    sentences_in_words = sentence_to_word(sentences)
    question_in_words = sentence_to_word([data_row["question"]])[0]

    max_overlap = None
    max_match_sentance_id = None

    question_words = set(question_in_words)
    for sentance_id in range(len(sentences_in_words)):
        sentence_words = set(sentences_in_words[sentance_id])
        overlap = len(sentence_words.intersection(question_words))
        if max_overlap is None or overlap > max_overlap:
            max_overlap = overlap
            max_match_sentance_id = sentance_id
            
            
    # do smth to extract answer
    return sentences[max_match_sentance_id], ' '.join(list(set(normalize_answer(sentences[max_match_sentance_id]).split()) - set(question_in_words)))

## load data

In [26]:
data = pd.read_csv("../../data/train_task1_latest.csv")
data_test = pd.read_csv("../../data/sdsj_A_test.csv")

## generate questions using patterns

**p4 - герерирование вопроса заполнением случайными словами структуры вопроса"**
87% при baseline features

In [27]:
pd.options.display.max_colwidth = 1000

In [28]:
morph = pymorphy2.MorphAnalyzer()

def get_patterns(text):
    tokens = text.split(' ')
    return [[p.tag.POS for p in morph.parse(word)][0] for word in tokens]

In [29]:
# создадим массив возможных вариаций частей речи в вопросе
df = pd.DataFrame(data.question.unique())
df.columns = ['question']
df['question_patterns'] = df['question'].map(lambda x: get_patterns(x.strip('?')))
data = data.merge(df, on = 'question', how = 'left')
data['question_patterns_str'] = data.question_patterns.map(lambda x: ' '.join([i for i in x if i is not None]))
unique_question_patterns = data.question_patterns_str.unique()
unique_question_patterns[0]

'PREP ADJF NOUN ADJF NOUN VERB PREP NOUN PREP NOUN ADJF CONJ NOUN NOUN NOUN CONJ PRCL NOUN'

In [30]:
def find_patterns(text, patterns):

    '''
    Tries to find pattern ([NOUN, VERB] ex.) in text and extract it. None if none found2
    '''
    
    text_patterns = get_patterns(text.strip('?'))
    text_patterns_str = ' '.join([i if i is not None else 'NONE' for i in get_patterns(text.strip('?'))])
    
    i_start = text_patterns_str.find(patterns)
    
    if i_start == -1:
        return None
    
    # count number of spaces before i_start -> number of words to skip
    word_start = text_patterns_str[:i_start].count(' ')
    wort_end = word_start + len(patterns.split(' '))
    
    return ' '.join(text.split(' ')[word_start:wort_end])

In [ ]:
data_generated = pd.DataFrame(columns=['paragraph_id', 'paragraph'])
data_generated['generated_question'] = ''

count = 0
for paragraph, paragraph_group in tqdm.tqdm(data.groupby('paragraph')):
    question_pool = paragraph_group.question.unique()
    paragraph_pool = np.append(paragraph.split('. '), question_pool)
    
    for i in range(0, np.random.randint(1, 5)):
    
        random_question_patterns = unique_question_patterns[np.random.randint(0, len(unique_question_patterns))]

        generated_question = ' '.join(question_pool[np.random.randint(0, len(question_pool))].split(' ')[:3]) + ' '
        found_patterns = []
        patterns = random_question_patterns.split(' ')[3:]
        for i in range(len(patterns)):

            if len(found_patterns) >= len(patterns[:i + 1]):
                continue

            failed = 0
            patterns_to_find = patterns[i:]
            while (len(patterns_to_find) != 0):

                # try to find patterns in paragraph / questions
                for sentence in paragraph_pool:
                    found_text = find_patterns(sentence,  ' '.join([i if i is not None else 'NONE' for i in patterns_to_find]))
                    if found_text is not None:
                        break

                # remember text if found
                if found_text is None:
                    failed += 1
                    patterns_to_find = patterns_to_find[:-1]
                    continue
                else:
                    generated_question += found_text + ' '
                    found_patterns.extend(patterns_to_find)
                    break


        generated_question += '?'
        
        if generated_question not in question_pool and len(generated_question.split(' ')) > 5:
                data_generated.loc[count] = [paragraph_group.paragraph_id.values[0], paragraph, generated_question]
                count += 1

    if count >= 20000:
        break    


 11%|█         | 1005/9078 [1:10:59<10:54:48,  4.87s/it]

In [38]:
data_generated = data_generated[data_generated['generated_question'] != '']
data_generated[['paragraph_id', 'paragraph', 'generated_question']].drop_duplicates().dropna().to_csv('../../data/taskA_generated_questions_p_6.csv', sep=';', index = False)

In [39]:
data_generated

,paragraph_id,paragraph,generated_question
0,7658,"Everybody , как и хотела Мадонна, выпускают синглом. При нулевом бюджете на раскрутку фото певицы решают не помещать на обложке, чтобы не отпугнуть цветную аудиторию якобы негритянской диско-соул-певицы . Everybody поднимается на 3-е место в чарте Hot Dance Club Songs, а потом на 107 место в основном, немного не дотянув до первой сотни Hot 100 журнала Billboard[91]. Менеджмент считает это отличным результатом, учитывая нулевые затраты на пиар, и хочет убедиться, что взлёт Everybody не случаен. По просьбе Мадонны вместо Каминса берут более опытного штатного аранжировщика Warner Bros. Records Регги Лукаса (англ.)русск.. Второй сингл Burning Up тоже достигает в чарте танцевальных хитов 3-го места, повторив успех Everybody . И только после этого Мадонне позволяют арендовать студию для записи первого альбома[91].",Почему при нулевом нулевом на раскрутку нулевом бюджете чарте танцевальных нулевом бюджете ?
1,7658,"Everybody , как и хотела Мадонна, выпускают синглом. При нулевом бюджете на раскрутку фото певицы решают не помещать на обложке, чтобы не отпугнуть цветную аудиторию якобы негритянской диско-соул-певицы . Everybody поднимается на 3-е место в чарте Hot Dance Club Songs, а потом на 107 место в основном, немного не дотянув до первой сотни Hot 100 журнала Billboard[91]. Менеджмент считает это отличным результатом, учитывая нулевые затраты на пиар, и хочет убедиться, что взлёт Everybody не случаен. По просьбе Мадонны вместо Каминса берут более опытного штатного аранжировщика Warner Bros. Records Регги Лукаса (англ.)русск.. Второй сингл Burning Up тоже достигает в чарте танцевальных хитов 3-го места, повторив успех Everybody . И только после этого Мадонне позволяют арендовать студию для записи первого альбома[91].",Какой альбом Мадонны нулевом бюджете ?
2,850,"Freewheelin’ Bob Dylan произвёл большое впечатление не только на любителей фолк-музыки и не только в США. Кумиры британской публики The Beatles, по свидетельству Джорджа Харрисона, заслушали его до дыр. Альбом Дилана научил ливерпульскую четвёрку, что содержанием популярной музыки могут быть не только романтические чувства мужчины и женщины. По примеру Дилана битлы стали экспериментировать с песнями и усложнять свой репертуар. Между тем в начале 1964 года выходит новый альбом протестных песен Дилана The Times They Are A-Changing , за которым через несколько месяцев последовал диск Another Side of Bob Dylan , в музыкальной ткани которого появляются вкрапления блюза и ритм-энд-блюза. Под влиянием чтения Джона Китса и Артюра Рембо слова песен Дилана становятся всё более литературными и суггестивными. Впервые в истории музыки суть не излагается прямым текстом, а передаётся сцеплениями сложных образов. Иногда слово следует за словом, будто поток сознания .",Под влиянием чего на любителей фолк-музыки на ?
3,850,"Freewheelin’ Bob Dylan произвёл большое впечатление не только на любителей фолк-музыки и не только в США. Кумиры британской публики The Beatles, по свидетельству Джорджа Харрисона, заслушали его до дыр. Альбом Дилана научил ливерпульскую четвёрку, что содержанием популярной музыки могут быть не только романтические чувства мужчины и женщины. По примеру Дилана битлы стали экспериментировать с песнями и усложнять свой репертуар. Между тем в начале 1964 года выходит новый альбом протестных песен Дилана The Times They Are A-Changing , за которым через несколько месяцев последовал диск Another Side of Bob Dylan , в музыкальной ткани которого появляются вкрапления блюза и ритм-энд-блюза. Под влиянием чтения Джона Китса и Артюра Рембо слова песен Дилана становятся всё более литературными и суггестивными. Впервые в истории музыки суть не излагается прямым текстом, а передаётся сцеплениями сложных образов. Иногда слово следует за словом, будто поток сознания .",Под влиянием чего впечатление на любителей популярной музыки могут ?
4,850,"Freewheelin’ Bob Dylan произвёл большое впечатление не только на любителей фолк-музыки и не тол

In [22]:
data_generated

,paragraph_id,paragraph,generated_question
0,7658,"Everybody , как и хотела Мадонна, выпускают синглом. При нулевом бюджете на раскрутку фото певицы решают не помещать на обложке, чтобы не отпугнуть цветную аудиторию якобы негритянской диско-соул-певицы . Everybody поднимается на 3-е место в чарте Hot Dance Club Songs, а потом на 107 место в основном, немного не дотянув до первой сотни Hot 100 журнала Billboard[91]. Менеджмент считает это отличным результатом, учитывая нулевые затраты на пиар, и хочет убедиться, что взлёт Everybody не случаен. По просьбе Мадонны вместо Каминса берут более опытного штатного аранжировщика Warner Bros. Records Регги Лукаса (англ.)русск.. Второй сингл Burning Up тоже достигает в чарте танцевальных хитов 3-го места, повторив успех Everybody . И только после этого Мадонне позволяют арендовать студию для записи первого альбома[91].",При каком бюджете арендовать студию для записи ?
1,7658,"Everybody , как и хотела Мадонна, выпускают синглом. При нулевом бюджете на раскрутку фото певицы решают не помещать на обложке, чтобы не отпугнуть цветную аудиторию якобы негритянской диско-соул-певицы . Everybody поднимается на 3-е место в чарте Hot Dance Club Songs, а потом на 107 место в основном, немного не дотянув до первой сотни Hot 100 журнала Billboard[91]. Менеджмент считает это отличным результатом, учитывая нулевые затраты на пиар, и хочет убедиться, что взлёт Everybody не случаен. По просьбе Мадонны вместо Каминса берут более опытного штатного аранжировщика Warner Bros. Records Регги Лукаса (англ.)русск.. Второй сингл Burning Up тоже достигает в чарте танцевальных хитов 3-го места, повторив успех Everybody . И только после этого Мадонне позволяют арендовать студию для записи первого альбома[91].",При каком бюджете певицы решают синглом ?
2,7658,"Everybody , как и хотела Мадонна, выпускают синглом. При нулевом бюджете на раскрутку фото певицы решают не помещать на обложке, чтобы не отпугнуть цветную аудиторию якобы негритянской диско-соул-певицы . Everybody поднимается на 3-е место в чарте Hot Dance Club Songs, а потом на 107 место в основном, немного не дотянув до первой сотни Hot 100 журнала Billboard[91]. Менеджмент считает это отличным результатом, учитывая нулевые затраты на пиар, и хочет убедиться, что взлёт Everybody не случаен. По просьбе Мадонны вместо Каминса берут более опытного штатного аранжировщика Warner Bros. Records Регги Лукаса (англ.)русск.. Второй сингл Burning Up тоже достигает в чарте танцевальных хитов 3-го места, повторив успех Everybody . И только после этого Мадонне позволяют арендовать студию для записи первого альбома[91].",Имя более опытного 3-е место в чарте ?
3,7658,"Everybody , как и хотела Мадонна, выпускают синглом. При нулевом бюджете на раскрутку фото певицы решают не помещать на обложке, чтобы не отпугнуть цветную аудиторию якобы негритянской диско-соул-певицы . Everybody поднимается на 3-е место в чарте Hot Dance Club Songs, а потом на 107 место в основном, немного не дотянув до первой сотни Hot 100 журнала Billboard[91]. Менеджмент считает это отличным результатом, учитывая нулевые затраты на пиар, и хочет убедиться, что взлёт Everybody не случаен. По просьбе Мадонны вместо Каминса берут более опытного штатного аранжировщика Warner Bros. Records Регги Лукаса (англ.)русск.. Второй сингл Burning Up тоже достигает в чарте танцевальных хитов 3-го места, повторив успех Everybody . И только после этого Мадонне позволяют арендовать студию для записи первого альбома[91].",Какой альбом Мадонны раскрутку фото певицы ?
4,850,"Freewheelin’ Bob Dylan произвёл большое впечатление не только на любителей фолк-музыки и не только в США. Кумиры британской публики The Beatles, по свидетельству Джорджа Харрисона, заслушали его до дыр. Альбом Дилана научил ливерпульскую четвёрку, что содержанием популярной музыки могут быть не только романтические чувства мужчины и женщины. По примеру Дилана битлы стали экспериментировать с песнями и усложнять свой репертуар. Между тем в начале 1964 года

In [ ]:
# data_generated = pd.DataFrame(columns=['paragraph_id', 'paragraph'])
# data_generated['generated_question'] = ''

# count = 0
# for paragraph, paragraph_group in tqdm.tqdm(data.groupby('paragraph')):
#     sententence_pool = np.append(paragraph_group.question.unique(), paragraph.split('. '))
#     sententence_pool_2 = np.append(paragraph.split('. '), paragraph_group.question.unique())
    
#     for i in range(0, np.random.randint(1, 5)):
    
#         random_question_patterns = unique_question_patterns[np.random.randint(0, len(unique_question_patterns))]

#         generated_question = ''
#         found_patterns = []
#         patterns = random_question_patterns.split(' ')
#         first = True
#         for i in range(len(patterns)):

#             if len(found_patterns) >= len(patterns[:i + 1]):
#                 continue

#             failed = 0
#             patterns_to_find = patterns[i:]
#             while (len(patterns_to_find) != 0):

#                 # if first - use questions first, else use paragraph first
#                 if first:
#                     first = False
#                     s_p = sententence_pool
#                 else:
#                     s_p = sententence_pool_2

#                 # try to find patterns in paragraph / questions
#                 for sentence in s_p:
#                     found_text = find_patterns(sentence,  ' '.join([i if i is not None else 'NONE' for i in patterns_to_find]))
#                     if found_text is not None:
#                         break

#                 # remember text if found
#                 if found_text is None:
#                     failed += 1
#                     patterns_to_find = patterns_to_find[:-1]
#                     continue
#                 else:
#                     generated_question += found_text + ' '
#                     found_patterns.extend(patterns_to_find)
#                     break


#         generated_question += '?'
#         data_generated.loc[count] = [paragraph_group.paragraph_id.values[0], paragraph, generated_question]
#         count += 1

#     if count >= 10000:
#         break
    

** p3 - генерирование повторяющейся стуктуры** - 95.1%

In [21]:
data_generated = pd.DataFrame(columns=data.columns)
data_generated['generated_question'] = ''

count = 0
for paragraph, paragraph_group in tqdm.tqdm(data.groupby('paragraph')):
    
    for row in paragraph_group.iterrows():
        question = row[1]['question'].strip('?').strip(' ')
        question_words = [x for x in question.split(' ')]
        if int(len(question_words) / 2) <= 1: 
            continue
        
        # pick chunk to swap
        word_to_swap = np.random.randint(1, int(len(question_words) / 2))
        index_to_swap = question.find(' ' + question_words[word_to_swap])

        # iterate through question pool
        questions_pool = paragraph_group[paragraph_group.question != row[1]['question']].question.values
        if len(questions_pool) == 0:
            continue
            
        time_to_generate = np.random.randint(2, 8)
        for i in range(time_to_generate):

            generated_question = question[:index_to_swap]
            time_to_swap = np.random.randint(2, 8)

            for i in range(time_to_swap):
                pool_question = questions_pool[np.random.randint(0, len(questions_pool))]
                pool_question_words = [x for x in pool_question.split(' ')]
                if len(pool_question_words) < 3: 
                    i -= 1
                    continue

                generated_question = generated_question.strip('?').strip(' ')

                # pick chunk to swap
                pool_word_to_swap = np.random.randint(1, len(pool_question_words) - 1)
                if pool_question_words[pool_word_to_swap] == '?':
                    continue 
                pool_index_to_swap = pool_question.find(' ' + pool_question_words[pool_word_to_swap])

                # swap
                generated_question += pool_question[pool_index_to_swap:]

            if generated_question != row[1]['question'] and len(generated_question) > len(question):
                data_generated.loc[count] = [row[1]['paragraph_id'], row[1]['question_id'], paragraph, row[1]['question'], 0, generated_question]
                count += 1

    if count >= 40000:
        break
        


  9%|▉         | 838/9078 [05:09<1:00:25,  2.27it/s] 

In [25]:
data_generated = data_generated[data_generated['generated_question'] != '']
data_generated[['paragraph_id', 'paragraph', 'generated_question']].drop_duplicates().dropna().to_csv('../data/taskA_generated_questions_p_3.csv', sep=';', index = False)

**p1 - соединение по слову** - 91.7%

In [64]:
patterns = ['ADJF+NOUN','PRTS+NOUN', 'NOUN+NOUN', 'NOUN+VERB', 'VERB+ADJF']
morph = pymorphy2.MorphAnalyzer()

def match(ngram, patterns = patterns):
    index = []
    for word in ngram:
        buf = [(p.normal_form, p.tag.POS) for p in morph.parse(word)]
        index.append((word,buf))
    pos_tagging = product(*[ind[1] for ind in index])
    possible_patterns = map(lambda pos_tag: list(zip(*pos_tag)), pos_tagging)
    possible_patterns = map(lambda pattern: [pattern[0], map(lambda grammeme: grammeme, pattern[1])], possible_patterns)
    possible_patterns = map(lambda pattern: (pattern[0], '+'.join(pattern[1])), possible_patterns)
    for pattern in possible_patterns:
        if pattern[1] in patterns:
            return pattern
    return None        

def get_pattern(text, patterns = patterns):
    tokens = [x for x in normalize(text)]
    pattern_coll2  = {}
    coll2 = nltk.FreqDist(nltk.bigrams(tokens))
    c2_index = -1
    for c2 in coll2:
        c2_index += 1
        try:
            p = match(c2, patterns)
            if p != None:
                collocation = ' '.join(p[0])
                if collocation not in pattern_coll2:
                    pattern_coll2[collocation] = [p[1], c2_index]
        except:
            pass

    return pattern_coll2

In [77]:
data_generated = pd.DataFrame(columns=data.columns)
data_generated['generated_question'] = ''

count = 0
for paragraph, paragraph_group in tqdm.tqdm(data.groupby('paragraph')):
    
    for row in paragraph_group.iterrows():
        question = row[1]['question'].strip('?').strip(' ')
        question_words = [x for x in question.split(' ') if len(x) > 3]
        if len(question_words) < 4:
            continue
        
        # iterate through question pool
        questions_pool = paragraph_group[paragraph_group.question != row[1]['question']].question.values
        if len(questions_pool) == 0:
            continue
            
        for pool_question in questions_pool:
        
            # pick chunk to swap
            word_to_swap = np.random.randint(1, len(question_words) - 2)
            if question_words[word_to_swap] == '?':
                continue 

            index_to_swap = question.find(' ' + question_words[word_to_swap])
        
            # swap
            generated_question = question[:index_to_swap]
            pool_index_to_swap = pool_question.find(' ' + question_words[word_to_swap])
            
            if pool_index_to_swap != -1:
                generated_question += pool_question[pool_index_to_swap:]
                    
                if generated_question != row[1]['question'] and len(generated_question) > len(question):
                    data_generated.loc[count] = [row[1]['paragraph_id'], row[1]['question_id'], paragraph, row[1]['question'], row[1]['answer'], generated_question]
                    count += 1
            
    if count >= 20000:
        break
        


100%|██████████| 9078/9078 [01:30<00:00, 100.67it/s]


In [ ]:
data_generated = data_generated[data_generated['generated_question'] != '']
data_generated[['paragraph_id', 'paragraph', 'generated_question']].drop_duplicates().dropna().to_csv('../data/taskA_generated_questions_p_1.csv', sep=';', index = False)